# Course Scheduler
A course scheduler developed for the machine learning course. 

This scheduler uses genetic algorithm to find the best possible course schedule.

## Usage

After running of all the coding the results are like:
  Best Fitness in Generation: 0.854654654655
  Best Fitness in Generation: 0.861661661662

  ...after 140 generations...

  Best Fitness in Generation: 0.97037037037
  Generation: 141
```

Output will be written to `{project-dir}/schedule_output.html` file. 


In [1]:
# mount Google Drive files by running the following code
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
""" Slot class represents timeslots in a week """
class Slot:
    def __init__(self, inputString):
        stringParts = inputString.split(",")
        cursorPosition = 0

        self.id = stringParts[cursorPosition].strip()
        cursorPosition += 1

        self.dow = stringParts[cursorPosition].strip()
        cursorPosition += 1

        self.time = stringParts[cursorPosition].strip()
        cursorPosition += 1

    def printObject(self):
        print(str(self.dow) + ", " + str(self.time))

In [3]:
""" Room class represents classrooms """
class Room:
    def __init__(self, inputString):
        stringParts = inputString.split(",")
        cursorPosition = 0

        self.id = stringParts[cursorPosition].strip()
        cursorPosition += 1

        self.name = stringParts[cursorPosition].strip()
        cursorPosition += 1

        self.capacity = stringParts[cursorPosition].strip()
        cursorPosition += 1

    def printObject(self):
        print("Room: " + str(self.id))
        print("Name: " + str(self.name))
        print("Capacity: " + str(self.capacity))

In [4]:
""" Course class represents courses offered by department """
class Course:
    def __init__(self, inputString):
        stringParts = inputString.split(",")
        cursorPosition = 0

        self.id = stringParts[cursorPosition].strip()
        cursorPosition += 1

        self.name = stringParts[cursorPosition].strip()
        cursorPosition += 1

        self.capacity = stringParts[cursorPosition].strip()
        cursorPosition += 1

        self.arrangement = stringParts[cursorPosition].strip()
        cursorPosition += 1

        self.instructorIndex = int(stringParts[cursorPosition].strip())
        cursorPosition += 1

    def printObject(self):
        print("Course Name: " + str(self.name) + str(" | ") + "Capacity: " + str(self.capacity))
        print("Arrangement: " + str(self.arrangement))


In [5]:
""" Instructor class represents faculty staff """
class Instructor:
    def __init__(self, inputString):
        stringParts = inputString.split(",")
        cursorPosition = 0

        self.id = stringParts[cursorPosition].strip()
        cursorPosition += 1

        self.name = stringParts[cursorPosition].strip()
        cursorPosition += 1

        self.unwanteds = []
        self.unpreferreds = []

        numberOfUnwanteds = int(stringParts[cursorPosition].strip())
        cursorPosition += 1

        for i in range(cursorPosition, cursorPosition + numberOfUnwanteds):
            self.unwanteds.append(int(stringParts[i]))

        cursorPosition += numberOfUnwanteds

        numberOfUnpreferreds = int(stringParts[cursorPosition].strip())
        for i in range(cursorPosition, cursorPosition + numberOfUnpreferreds):
            self.unpreferreds.append(int(stringParts[i]))

    def wantsSlot(self, slotId):
        if slotId in self.unwanteds:
            return False

        return True

    def prefersSlot(self, slotId):
        if slotId in self.unpreferreds:
            return False

        return True

    def printObject(self):
        print("Instructor Name: " + str(self.name))

    def getInitials(self):
        return '.'.join(name[0].upper() for name in self.name.split())


In [6]:
import random
from operator import itemgetter

# from Slot import Slot
# from Room import Room
# from Course import Course
# from Instructor import Instructor
# from Schedule import Schedule

class GeneticAlgorithm:
    def __init__(self, slots, rooms, courses, instructors):
        self.slots = slots
        self.rooms = rooms
        self.courses = courses
        self.instructors = instructors

        # possible solutions
        self.chromosomes = []

        # new generations generated on every turn
        self.newChromosomes = []
        self.newChromosomeCount = 80

        self.lastFitness = 0
        self.fitnessConstantSince = 0

    def initChromosomes(self, numberOfChromosomes):
        for i in range(0, numberOfChromosomes):
            chromosome = Schedule(self.slots, self.rooms, self.courses, self.instructors)
            chromosome.createSchedule()

            self.chromosomes.append({"chromosome": chromosome, "fitness": chromosome.calculateFitness()})

    def continueIteration(self):
        output="/content/drive/My Drive/Python-Examples-Prof. salwani/6. Genetic Algorithm/schedule_output.html"
        for chromosome in self.chromosomes:
            if chromosome["fitness"] >= 0.90 and chromosome["chromosome"].satisfactory:
                chromosome["chromosome"].printObject()
                chromosome["chromosome"].saveHtml(output)
                return False

        print("Best Fitness in Generation: " + str(self.chromosomes[len(self.chromosomes) - 1]["fitness"]))
        if self.lastFitness == self.chromosomes[len(self.chromosomes) - 1]["fitness"]:
            self.fitnessConstantSince += 1
            if self.fitnessConstantSince >= 100:
                self.chromosomes[len(self.chromosomes) - 1]["chromosome"].printObject()
                self.chromosomes[len(self.chromosomes) - 1]["chromosome"].saveHtml(output)
                print("Fitness score does not seem improve anymore. This schedule may not satify some requirements but a \"significant\" time is passed, so it is returned. Please try again with (smaller) inputs")
                return False
        else:
            self.lastFitness = self.chromosomes[len(self.chromosomes) - 1]["fitness"]
            self.fitnessConstantSince = 0

        return True

    def execute(self):
        while self.continueIteration():
            # new generation of solutions
            self.newChromosomes = []

            for mutationIndex in range(int(len(self.chromosomes) / 2), len(self.chromosomes)):
                if random.randint(0, 5) == 4: # mutation probablity 20%
                    p4 = self.chromosomes[mutationIndex]["chromosome"].mutation()
                    self.newChromosomes.append({"chromosome": p4, "fitness": p4.calculateFitness()})

            # select random pair as parent for new generation
            selectedPairs = []
            while len(selectedPairs) < (self.newChromosomeCount - len(self.newChromosomes)) / 2:
                pair = (random.randint(0, len(self.chromosomes) - 1), random.randint(len(self.chromosomes) * 0.90, len(self.chromosomes) - 1))
                if pair[0] != pair[1]:
                    selectedPairs.append(pair)

            for pair in selectedPairs:
                if random.randint(0, 5) == 4: # mutation probablity 20%
                    children = self.chromosomes[pair[0]]["chromosome"].crossover(self.chromosomes[pair[1]]["chromosome"])
                    for nc in children:
                        nc.rebuildSlots()
                        self.newChromosomes.append({"chromosome": nc, "fitness": nc.calculateFitness()})

            self.chromosomes = sorted(self.chromosomes, key=itemgetter('fitness'))

            for i in range(0, min(len(self.newChromosomes), int(len(self.chromosomes) * 0.80))): # dont change best 20% chromosomes
                self.chromosomes[i] = self.newChromosomes[i]

In [7]:
# from src.Room import Room
# from src.Course import Course
# from src.Instructor import Instructor

class CourseClass:
    def __init__(self, course, instructor):
        self.course = course
        self.instructor = instructor

    def roomHasEnoughCapacity(self, room):
        capacity = {
            "Large": 3,
            "Medium": 2,
            "Small": 1
        }

        if capacity[room.capacity] > capacity[self.course.capacity]:
            return 2
        elif capacity[room.capacity] == capacity[self.course.capacity]:
            return 8

        return 0

    def printObject(self):
        self.course.printObject()
        self.instructor.printObject()

In [8]:
# from CourseClass import CourseClass

import random
import copy

class Schedule:
    def __init__(self, slots, rooms, courses, instructors):
        self.classes = []
        self.slots = []
        for i in range(0, len(slots) * len(rooms)):
            self.slots.append([])

        self.rooms = rooms
        self.fitness = 0.0
        self.satisfactory = False

        self.courses = courses
        self.timeSlots = slots
        self.instructors = instructors
        self.generation = 1

    def createSchedule(self):
        for course in self.courses:
            courseClass = CourseClass(course, self.instructors[course.instructorIndex - 1])

            if course.arrangement == "A3":
                slotIndex = random.randint(0, len(self.slots) - 3)

                self.slots[slotIndex].append(courseClass)
                self.slots[slotIndex + 1].append(courseClass)
                self.slots[slotIndex + 2].append(courseClass) # assuming all courses are A3

                self.classes.append({"class": courseClass, "slotIndex": slotIndex, "length": 3})
            else:
                slotIndex = random.randint(0, len(self.slots) - 2)

                self.slots[slotIndex].append(courseClass)
                self.slots[slotIndex + 1].append(courseClass)

                self.classes.append({"class": courseClass, "slotIndex": slotIndex, "length": 2})

                slotIndex = random.randint(0, len(self.slots) - 1)

                self.slots[slotIndex].append(courseClass)

                self.classes.append({"class": courseClass, "slotIndex": slotIndex, "length": 1})

    def instructorAvailable(self, instructor, timeSlot):
        checkSlot = timeSlot - len(self.slots)

        while checkSlot > 0:
            if self.slots[checkSlot] != [] and self.slots[checkSlot][0].instructor.id == instructor.id: # two class may occur at same time
                return False

            checkSlot -= len(self.slots)

        checkSlot = timeSlot + len(self.slots)

        while checkSlot < len(self.slots):
            if self.slots[checkSlot] != [] and self.slots[checkSlot].instructor.id == instructor.id:
                return False

            checkSlot += len(self.slots)

        return True

    def calculateFitness(self):
        coursesToAdd = []
        extraCourses = []
        for course in self.courses:
            coursesToAdd.append(course.id)
            if course.arrangement == 'A21':
                coursesToAdd.append(course.id)

        self.satisfactory = None

        classPoints = []
        classPoint=0
        for someClass in self.classes:
            classIndex = someClass["slotIndex"]

            spareRoom = False
            if someClass["length"] == 3:
                if len(self.slots[classIndex]) == 1 and len(self.slots[classIndex + 1]) == 1 and len(self.slots[classIndex + 2]) == 1:
                    spareRoom = True
            elif someClass["length"] == 2:
                if len(self.slots[classIndex]) == 1 and len(self.slots[classIndex + 1]) == 1:
                    spareRoom = True
            elif someClass["length"] == 1:
                if len(self.slots[classIndex]) == 1:
                    spareRoom = True
            else:
                raise ValueError(someClass)

            if spareRoom:
                # class is using a spare room
                classPoint = classPoint+4
            else:
                self.satisfactory = False

            room = classIndex / len(self.timeSlots)
            roomIndex = room
            room = self.rooms[int(roomIndex)]

            roomCapacity = self.slots[classIndex][0].roomHasEnoughCapacity(room)
            if roomCapacity:
                classPoint = classPoint+roomCapacity
            else:
                self.satisfactory = False

            if self.instructorAvailable(self.slots[classIndex][0].instructor, classIndex):
                classPoint = classPoint+4
            else:
                self.satisfactory = False

            if self.slots[classIndex][0].instructor.wantsSlot((classIndex % len(self.timeSlots)) + 1):
                classPoint = classPoint+4
            else:
                self.satisfactory = False

            if self.slots[classIndex][0].instructor.prefersSlot((classIndex % len(self.timeSlots)) + 1):
                classPoint = classPoint+1

            dayIndex = (classIndex % len(self.timeSlots)) / (len(self.timeSlots) / 5)
            if dayIndex != ((classIndex + someClass["length"] - 1) % len(self.timeSlots)) / (len(self.timeSlots) / 5):
                self.satisfactory = False
            else:
                classPoint = classPoint+4

            # check if same course arranged for the different classrooms at the same time
            falseFlag = False

            hourIndex = classIndex - dayIndex * 9 - roomIndex * 45
            checkIndex = hourIndex
            while checkIndex < len(self.slots):
                if checkIndex == classIndex:
                    checkIndex += 45
                    continue

                if len(self.slots[int(checkIndex)]) and self.slots[int(checkIndex)][0].course.id == self.slots[int(classIndex)][0].course.id:
                    falseFlag = True
                    self.satisfactory = False
                    break

                checkIndex += 45

            if falseFlag != True:
                classPoint = classPoint+4

            classPoints.append(classPoint)

            if someClass["class"].course.id in coursesToAdd:
                coursesToAdd.remove(someClass["class"].course.id)
            else:
                extraCourses.append(someClass["class"].course.id)

        if len(coursesToAdd) == 0 and len(extraCourses) == 0:
            for i in range(0, len(classPoints)):
                classPoints[i] += 16

        if len(self.classes):
            self.fitness = (sum(classPoints) * 1.0) / (len(classPoints) * 45)
        else:
            self.fitness = 0

        if self.satisfactory is None:
            if len(coursesToAdd) or len(extraCourses):
                self.satisfactory = False
            else:
                self.satisfactory = True

        return self.fitness

    def rebuildSlots(self):
        slotsLen = len(self.slots)

        self.slots = []
        for i in range(0, slotsLen):
            self.slots.append([])

        for someClass in self.classes:
            if someClass["length"] == 3:
                self.slots[someClass["slotIndex"]].append(someClass["class"])
                self.slots[someClass["slotIndex"] + 1].append(someClass["class"])
                self.slots[someClass["slotIndex"] + 2].append(someClass["class"])
            elif someClass["length"] == 2:
                self.slots[someClass["slotIndex"]].append(someClass["class"])
                self.slots[someClass["slotIndex"] + 1].append(someClass["class"])
            elif someClass["length"] == 1:
                self.slots[someClass["slotIndex"]].append(someClass["class"])
            else:
                print("Hata")

    def crossover(self, p2):
        random.shuffle(self.classes)
        random.shuffle(p2.classes)

        firstCrossoverIndex = random.randint(0, len(self.classes) - 1)
        secondCrossoverIndex = random.randint(firstCrossoverIndex, len(self.classes) - 1)

        firstChild = Schedule(self.timeSlots, self.rooms, self.courses, self.instructors)
        firstChild.classes = self.classes[0:firstCrossoverIndex] + p2.classes[firstCrossoverIndex:secondCrossoverIndex] + self.classes[secondCrossoverIndex:]
        firstChild.generation = max(self.generation, p2.generation) + 1

        secondChild = Schedule(self.timeSlots, self.rooms, self.courses, self.instructors)
        secondChild.classes = p2.classes[0:firstCrossoverIndex] + self.classes[firstCrossoverIndex:secondCrossoverIndex] + p2.classes[secondCrossoverIndex:]
        secondChild.generation = max(self.generation, p2.generation) + 1

        return [firstChild, secondChild]

    def mutation(self):
        p4 = copy.deepcopy(self)

        classIndex = random.randint(0, len(p4.classes) - 1)
        p4.classes[classIndex]["slotIndex"] = random.randint(0, len(p4.slots) - 3)
        p4.generation = self.generation + 1
        p4.rebuildSlots()

        return p4

    def saveHtml(self, filename):
        with open('/content/drive/My Drive/Python-Examples-Prof. salwani/6. Genetic Algorithm/data/template.html', 'r') as content_file:
            htmlRepresentation = ""

            htmlRepresentation += "<h1>Generation: " + str(self.generation) + " / " + " Fitness: " + str(self.fitness) + "</h1>"

            for day in range(0, 5):
                htmlRepresentation = htmlRepresentation + "<div class=\"row\">"
                htmlRepresentation = htmlRepresentation + "<h3>" + ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"][day] + "</h3>"
                htmlRepresentation = htmlRepresentation + "<table class=\"table table-bordered\">"

                htmlRepresentation += "<tr>"
                htmlRepresentation += "<th width=\"10%\">Room</th>"
                htmlRepresentation += "<th width=\"10%\">8:40</th>"
                htmlRepresentation += "<th width=\"10%\">9:40</th>"
                htmlRepresentation += "<th width=\"10%\">10:40</th>"
                htmlRepresentation += "<th width=\"10%\">11:40</th>"
                htmlRepresentation += "<th width=\"10%\">12:40</th>"
                htmlRepresentation += "<th width=\"10%\">13:40</th>"
                htmlRepresentation += "<th width=\"10%\">14:40</th>"
                htmlRepresentation += "<th width=\"10%\">15:40</th>"
                htmlRepresentation += "<th width=\"10%\">16:40</th>"
                htmlRepresentation += "</tr>"

                for room in range(0, len(self.rooms)):
                    htmlRepresentation = htmlRepresentation + "<tr>"

                    roomName = self.rooms[room].name
                    htmlRepresentation = htmlRepresentation + "<td>" + roomName + "</td>"

                    for hour in range(0, 9):
                        timeSlot = day * 9 + room * 45 + hour
                        if(len(self.slots[timeSlot])):
                            htmlRepresentation = htmlRepresentation + "<td>" + self.slots[timeSlot][0].course.name + " [" + self.slots[timeSlot][0].instructor.getInitials() +"]</td>"
                        else:
                            htmlRepresentation = htmlRepresentation + "<td></td>"

                    htmlRepresentation = htmlRepresentation + "</tr>"

                htmlRepresentation = htmlRepresentation + "</table>"
                htmlRepresentation = htmlRepresentation + "</div>"
                htmlRepresentation = htmlRepresentation + "<hr>"

            fileContents = content_file.read()

            fileContents = fileContents.replace('##PYTHON_CHANGE_THIS##', htmlRepresentation)

            writeToFile = open(filename,'w')
            writeToFile.write(fileContents)
            writeToFile.close()

    def printObject(self):
        print("Generation: " + str(self.generation))
        for i in range(0, len(self.slots)):
            roomIndex = int(i / len(self.timeSlots))
            timeIndex = i % len(self.timeSlots)

            if i % len(self.timeSlots) == 0:
                print("----- &&& -----")
                self.rooms[int(roomIndex)].printObject()
                print("")

            scheduledClasses = self.slots[i]
            if len(scheduledClasses):
                self.timeSlots[timeIndex].printObject()

            for scheduledClass in scheduledClasses:
                if scheduledClass is not None:
                    scheduledClass.printObject()
                    print("")


In [9]:
import sys
import random

# from src.Slot import Slot
# from src.Room import Room
# from src.Course import Course
# from src.Instructor import Instructor

# from src.GeneticAlgorithm import GeneticAlgorithm

def main(cmdArguments):
  if len(cmdArguments) <= 0:
      raise ValueError("Usage: python main.py <inputfile>")

  slots = []
  rooms = []
  courses = []
  instructors = []

  with open(cmdArguments) as fp:
      for line in fp:
          if line.startswith("#"):
              # ignore comments in input file
              continue

          lineSections = line.split("=")
          if lineSections[0].strip() == "Slot":
              slots.append(Slot(lineSections[1]))
          elif lineSections[0].strip() == "Room":
              rooms.append(Room(lineSections[1]))
          elif lineSections[0].strip() == "Course":
              courses.append(Course(lineSections[1]))
          elif lineSections[0].strip() == "Instructor":
              instructors.append(Instructor(lineSections[1]))

  algo = GeneticAlgorithm(slots, rooms, courses, instructors)
  algo.initChromosomes(100)
  algo.execute()

"""
    for course in courses:
        course.printObject()

    for room in rooms:
        room.printObject()

    for slot in slots:
        slot.printObject()

    for ins in instructors:
        ins.printObject()
"""

if __name__ == '__main__':
    # main(sys.argv[1:])
    arg= "/content/drive/My Drive/Python-Examples-Prof. salwani/6. Genetic Algorithm/data/SampleInput1.in"
    main(arg)


Best Fitness in Generation: 25.238038038038038
Best Fitness in Generation: 26.89069069069069
Best Fitness in Generation: 26.89069069069069
Best Fitness in Generation: 26.89069069069069
Best Fitness in Generation: 26.89069069069069
Best Fitness in Generation: 26.89069069069069
Best Fitness in Generation: 26.89069069069069
Best Fitness in Generation: 26.89069069069069
Best Fitness in Generation: 26.89069069069069
Best Fitness in Generation: 26.89069069069069
Best Fitness in Generation: 26.89069069069069
Best Fitness in Generation: 26.89069069069069
Best Fitness in Generation: 26.89069069069069
Best Fitness in Generation: 26.89069069069069
Best Fitness in Generation: 26.89069069069069
Best Fitness in Generation: 26.89069069069069
Best Fitness in Generation: 26.89069069069069
Best Fitness in Generation: 26.89069069069069
Best Fitness in Generation: 26.89069069069069
Best Fitness in Generation: 26.8980980980981
Best Fitness in Generation: 26.9993993993994
Best Fitness in Generation: 26.9993